In [6]:
import json
import networkx as nx

with open('NFA_mt.json') as _:
    NFA = json.load(_)
Graph = nx.Graph()
startingState = NFA['startingState']

In [7]:
def add_epsilon_moves(queue: list) -> list:
    visited = []

    while queue:
        state = queue.pop(0) 
        visited.append(state)
        if 'epsilon' in NFA[state]:
            epsilon_value = NFA[state]['epsilon']
            if isinstance(epsilon_value, list):
                for item in epsilon_value:
                    if item not in visited and item not in queue:
                        queue.append(item)
            else:
                if epsilon_value not in visited and epsilon_value not in queue:
                    queue.append(epsilon_value)
    return visited

def get_possible_inputs(states: set) -> set:
    inputs = set()
    for state in states:
        keys = list(NFA[state].keys())
        for key in keys[1:]:
            if key != 'epsilon' and key not in inputs:
                inputs.add(key)
    return inputs

def add_inputs(states: set, inputs: set) -> dict :
    state_inputs = {}
    for input in inputs:
        ns = set()
        for state in states:
            if(input in NFA[state]):
                ns.add(NFA[state][input])
        ns.update(add_epsilon_moves(list(ns)))
        state_inputs[input] = ns
    return state_inputs


def checkTerminating(states: set) -> bool:
    for state in states:
        if NFA[state]['isTerminatingState']:
            return True
    return False

In [8]:
states = set(); states_dict = {}
states.add(startingState)
ct = 0; nname = f"NS{ct}"
states.update(add_epsilon_moves([startingState]))

states_dict['startingState'] = nname
states_dict[nname] = {'isTerminatingState': checkTerminating(states)}; 
states_dict[nname]['states'] = states; ct+=1

kv = list(states_dict.keys())
i = 0
while i < len(kv):
    ns = kv[i]
    ep_states = set().clear()
    if ns == "startingState":
        i += 1
        continue
    ep_states = add_epsilon_moves(list(states_dict[ns].get('states')))
    inputs = get_possible_inputs(ep_states)
    outStates = add_inputs(ep_states, inputs)
    for input in inputs:
        for key in list(states_dict.keys())[1:]:
            if(states_dict[key].get('states') == outStates[input]):
                nname = key
                break
        else:   #Only enter if not breaks
            nname = f"NS{ct}"
            states_dict[nname] = {'isTerminatingState': checkTerminating(outStates[input])}
            states_dict[nname]['states'] = outStates[input]
            ct+=1
        states_dict[ns][input] = nname
    kv = list(states_dict.keys())
    i += 1

states_dict

{'startingState': 'NS0',
 'NS0': {'isTerminatingState': False,
  'states': {'S1', 'S3', 'S5', 'S7', 'S8', 'S9'},
  'b': 'NS1',
  'a': 'NS2'},
 'NS1': {'isTerminatingState': False,
  'states': {'S1', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9'},
  'b': 'NS1',
  'a': 'NS2'},
 'NS2': {'isTerminatingState': True,
  'states': {'S1',
   'S10',
   'S11',
   'S13',
   'S15',
   'S17',
   'S18',
   'S2',
   'S3',
   'S5',
   'S6',
   'S7',
   'S8',
   'S9'},
  'b': 'NS3',
  'a': 'NS4'},
 'NS3': {'isTerminatingState': True,
  'states': {'S1',
   'S14',
   'S16',
   'S18',
   'S3',
   'S4',
   'S5',
   'S6',
   'S7',
   'S8',
   'S9'},
  'b': 'NS1',
  'a': 'NS2'},
 'NS4': {'isTerminatingState': True,
  'states': {'S1',
   'S10',
   'S11',
   'S12',
   'S13',
   'S15',
   'S16',
   'S17',
   'S18',
   'S2',
   'S3',
   'S5',
   'S6',
   'S7',
   'S8',
   'S9'},
  'b': 'NS3',
  'a': 'NS4'}}

In [9]:
Graph = nx.DiGraph(startingState = nname); states = set(); 
ct = 0; nname = f"NS{ct}"
states.add(startingState)
states.update(add_epsilon_moves([startingState]))
Graph.add_node(nname, isTerminatingState= checkTerminating(states), states=states); ct+=1

graph_kv = list(Graph.nodes); i = 0
while i < len(graph_kv):
    ns = graph_kv[i]
    ep_states = set().clear()
    ep_states = add_epsilon_moves(list(Graph.nodes[ns]['states']))
    inputs = get_possible_inputs(ep_states)
    outStates = add_inputs(ep_states, inputs)
    for input in inputs:
        for key in list(Graph.nodes):
            if(Graph.nodes[key]['states'] == outStates[input]):
                nname = key
                break
        else:   #Only enter if not breaks
            nname = f"NS{ct}"
            Graph.add_node(nname, isTerminatingState= checkTerminating(outStates[input]), 
                           states = outStates[input])
            ct+=1
        Graph.add_edge(ns,nname,input = input)
    graph_kv = list(Graph.nodes)
    i += 1


print(Graph.graph)
print(Graph.nodes(data=True))
print(Graph.edges(data=True))

{'startingState': 'NS4'}
[('NS0', {'isTerminatingState': False, 'states': {'S3', 'S1', 'S5', 'S8', 'S7', 'S9'}}), ('NS1', {'isTerminatingState': False, 'states': {'S3', 'S6', 'S1', 'S5', 'S8', 'S7', 'S4', 'S9'}}), ('NS2', {'isTerminatingState': True, 'states': {'S15', 'S2', 'S6', 'S3', 'S1', 'S13', 'S5', 'S11', 'S18', 'S17', 'S7', 'S10', 'S8', 'S9'}}), ('NS3', {'isTerminatingState': True, 'states': {'S3', 'S6', 'S1', 'S14', 'S16', 'S5', 'S18', 'S7', 'S8', 'S4', 'S9'}}), ('NS4', {'isTerminatingState': True, 'states': {'S15', 'S2', 'S6', 'S3', 'S1', 'S13', 'S5', 'S16', 'S11', 'S18', 'S17', 'S12', 'S10', 'S7', 'S8', 'S9'}})]
[('NS0', 'NS1', {'input': 'b'}), ('NS0', 'NS2', {'input': 'a'}), ('NS1', 'NS1', {'input': 'b'}), ('NS1', 'NS2', {'input': 'a'}), ('NS2', 'NS3', {'input': 'b'}), ('NS2', 'NS4', {'input': 'a'}), ('NS3', 'NS1', {'input': 'b'}), ('NS3', 'NS2', {'input': 'a'}), ('NS4', 'NS3', {'input': 'b'}), ('NS4', 'NS4', {'input': 'a'})]
